# Comprehensive optimization in single qubit system (single parameter)

This is the source codes for the example discussed in Sec. VII in Ref. [1].

A single qubit system whose free evolution Hamiltonian is $H_0 = \frac{1}{2}\omega \sigma_3$ with 
$\omega$ the frequency and $\sigma_3$ a Pauli matrix. The dynamics of the system is governed by
\begin{align}
\partial_t\rho=-i[H_0, \rho]+ \gamma_{+}\left(\sigma_{+}\rho\sigma_{-}-\frac{1}{2}\{\sigma_{-}\sigma_{+},\rho\}\right)+ \gamma_{-}\left(\sigma_{-}\rho\sigma_{+}-\frac{1}{2}\{\sigma_{+}\sigma_{-},\rho\}\right),
\end{align}

where $\gamma_{+}$, $\gamma_{-}$ are decay rates and $\sigma_{\pm}=(\sigma_1 \pm \sigma_2)/2$. The control Hamiltonian
\begin{align}
H_\mathrm{c}=u_1(t)\sigma_1+u_2(t)\sigma_2+u_3(t)\sigma_3
\end{align}

with $u_i(t)$ $(i=1,2,3)$ the control field. Here $\sigma_{1}$, $\sigma_{2}$ are also Pauli matrices.

In this case, we consider two types of comprehensive optimization, the first one is optimization of probe state and control (SC), and the other is optimization of probe state, control and measurement (SCM). QFI is taken as the target function for SC and CFI for SCM.

[1] M. Zhang et al., QuanEstimation: an open-source toolkit for quantum parameter estimation,
arXiv:2205.15588.

In [1]:
from quanestimation import *
import numpy as np

# initial state
rho0 = 0.5*np.array([[1., 1.], [1., 1.]])
# free Hamiltonian
omega = 1.0
sx = np.array([[0., 1.], [1., 0.]])
sy = np.array([[0., -1.j], [1.j, 0.]]) 
sz = np.array([[1., 0.], [0., -1.]])
H0 = 0.5*omega*sz
# derivative of the free Hamiltonian on omega
dH = [0.5*sz]
# control Hamiltonians 
Hc = [sx,sy,sz]
# dissipation
sp = np.array([[0., 1.], [0., 0.]])  
sm = np.array([[0., 0.], [1., 0.]]) 
decay = [[sp, 0.], [sm, 0.1]]
# measurement
M1 = 0.5*np.array([[1., 1.], [1., 1.]])
M2 = 0.5*np.array([[1., -1.], [-1., 1.]])
M = [M1, M2]
M_num = 2
# time length for the evolution
tspan = np.linspace(0., 10., 2500)

## State and measurement optimization

In [4]:
# comprehensive optimization algorithm: DE
DE_paras = {"p_num":10, "psi0":[], "ctrl0":[], "measurement0":[], \
            "max_episode":1000, "c":1.0, "cr":0.5, "seed":1234}
com = ComprehensiveOpt(savefile=False, method="DE", **DE_paras)
com.dynamics(tspan, H0, dH, decay=decay, dyn_method="expm")
com.SM()

# load the measurements
M = np.load("measurements.npy")[-1]

# # comprehensive optimization algorithm: PSO
# PSO_paras = {"p_num":10, "psi0":[], "ctrl0":[], \
#              "measurement0":[], "max_episode":[1000,100], "c0":1.0, \
#              "c1":2.0, "c2":2.0, "seed":1234}
# com = ComprehensiveOpt(savefile=False, method="PSO", **PSO_paras)
# com.dynamics(tspan, H0, dH, decay=decay, dyn_method="expm")
# com.SM()
# # load the measurements
# M = np.load("measurements.npy")[-1]

comprehensive optimization
single parameter scenario
optimization algorithm: Differential Evolution (DE)
initial value of CFI is 3.201657
Iteration over, data saved.44 (999 episodes)
Final value of CFI is 36.787944


## State and control optimization

In [2]:
# comprehensive optimization algorithm: AD
AD_paras = {"Adam":False, "psi0":[], "ctrl0":[], "measurement0":[], \
            "max_episode":300, "epsilon":0.01, "beta1":0.90, "beta2":0.99}
com = ComprehensiveOpt(savefile=False, method="AD", **AD_paras)
com.dynamics(tspan, H0, dH, Hc=Hc, decay=decay, ctrl_bound=[-2.0,2.0], dyn_method="expm")
com.SC(W=[], target="QFIM", LDtype="SLD")

# # comprehensive optimization algorithm: DE
# DE_paras = {"p_num":10, "psi0":[], "ctrl0":[], "measurement0":[], \
#             "max_episode":1000, "c":1.0, "cr":0.5, "seed":1234}
# com = ComprehensiveOpt(savefile=False, method="DE", **DE_paras)
# com.dynamics(tspan, H0, dH, Hc=Hc, decay=decay, ctrl_bound=[-2.0,2.0], dyn_method="expm")
# com.SC(W=[], target="QFIM", LDtype="SLD")

# # comprehensive optimization algorithm: PSO
# PSO_paras = {"p_num":10, "psi0":[], "ctrl0":[], \
#              "measurement0":[], "max_episode":[1000,100], "c0":1.0, \
#              "c1":2.0, "c2":2.0, "seed":1234}
# com = ComprehensiveOpt(savefile=False, method="PSO", **PSO_paras)
# com.dynamics(tspan, H0, dH, Hc=Hc, decay=decay, ctrl_bound=[-2.0,2.0], dyn_method="expm")
# com.SC(W=[], M=M, target="CFIM")

comprehensive optimization
single parameter scenario
optimization algorithm: Automatic Differentiation (AD)
non-controlled value of QFI is 17.233383
initial value of QFI is 26.076300
Iteration over, data saved.66 (299 episodes)
Final value of QFI is 44.751166


## control and measurement optimization

In [3]:
# comprehensive optimization algorithm: DE
DE_paras = {"p_num":10, "psi0":[], "ctrl0":[], "measurement0":[], \
            "max_episode":1000, "c":1.0, "cr":0.5, "seed":1234}
com = ComprehensiveOpt(savefile=False, method="DE", **DE_paras)
com.dynamics(tspan, H0, dH, Hc=Hc, decay=decay, ctrl_bound=[-2.0,2.0], dyn_method="expm")
com.CM(rho0)
# load the measurements
M = np.load("measurements.npy")[-1]

# # comprehensive optimization algorithm: PSO
# PSO_paras = {"p_num":10, "psi0":[], "ctrl0":[], \
#              "measurement0":[], "max_episode":[1000,100], "c0":1.0, \
#              "c1":2.0, "c2":2.0, "seed":1234}
# com = ComprehensiveOpt(savefile=False, method="PSO", **PSO_paras)
# com.dynamics(tspan, H0, dH, Hc=Hc, decay=decay, ctrl_bound=[-2.0,2.0], dyn_method="expm")
# com.CM(rho0)
# # load the measurements
# M = np.load("measurements.npy")[-1]

comprehensive optimization
single parameter scenario
optimization algorithm: Differential Evolution (DE)
initial value of CFI is 10.460080
Iteration over, data saved.53 (999 episodes)
Final value of CFI is 43.443153


FileNotFoundError: measurements.csv not found.

## State, control and measurement optimization

In [ ]:
# comprehensive optimization algorithm: DE
DE_paras = {"p_num":10, "psi0":[], "ctrl0":[], "measurement0":[], \
            "max_episode":1000, "c":1.0, "cr":0.5, "seed":1234}
com = ComprehensiveOpt(savefile=False, method="DE", **DE_paras)
com.dynamics(tspan, H0, dH, Hc=Hc, decay=decay, ctrl_bound=[-2.0,2.0], dyn_method="expm")
com.SCM()
# load the measurements
M = np.load("measurements.npy")[-1]

# # comprehensive optimization algorithm: PSO
# PSO_paras = {"p_num":10, "psi0":[], "ctrl0":[], \
#              "measurement0":[], "max_episode":[1000,100], "c0":1.0, \
#              "c1":2.0, "c2":2.0, "seed":1234}
# com = ComprehensiveOpt(savefile=False, method="PSO", **PSO_paras)
# com.dynamics(tspan, H0, dH, Hc=Hc, decay=decay, ctrl_bound=[-2.0,2.0], dyn_method="expm")
# com.SCM()
# # load the measurements
# M = np.load("measurements.npy")[-1]